In [85]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# LCMS Map Assemblage

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/6-Map_Assemblage.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>


## Overview


This notebook teaches how to take raw model outputs and assemble final map classes


### Objective

In this tutorial, you learn how to manipulate raw model output GEE image arrays to create map outputs of a single class.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Looking at raw GEE image array model output assets
- Manipulating the image arrays for a basic map assemblage
- Performing a more complicated map assemblage that balances omission and commission for Change

In [1]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p
import inspect,operator
import matplotlib.pyplot as plt
import pandas as pd  
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

# Can set the port used for viewing map outputs
Map.port = 1235
print('Done')


Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
PyTables is not installed. No support for HDF output.
Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [2]:
workbench_url = 'https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [3]:
# Bring in all folders/collections that are needed
# These must already exist as they are created in previous notebooks
export_rawLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-5_rawLCMSOutputs'

export_assembledLCMSOutputs_collection = f'{export_path_root}/lcms-training_module-6_assembledLCMSOutputs'


aml.create_asset(export_assembledLCMSOutputs_collection, asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_assembledLCMSOutputs_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-6_assembledLCMSOutputs']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs
New asset projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs created
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-6_assembledLCMSOutputs
Done


In [59]:
Map.proxy_url = workbench_url
Map.clearMap()
# Bring in raw LCMS model outputs
raw_lcms = ee.ImageCollection(export_rawLCMSOutputs_collection)

# Bring in existing LCMS data for the class names, numbers, and colors
lcms_viz_dict = ee.ImageCollection("USFS/GTAC/LCMS/v2020-6").first().toDictionary().getInfo()
  
    
# Get some info
raw_lcms = raw_lcms.map(lambda img:img.set('product',img.bandNames().get(0)))
products = raw_lcms.aggregate_histogram('product').keys().getInfo()
products = ['Land_Cover','Land_Use','Change']

# Specify missing land cover codes in PRUSVI
# Talls Shrubs (2 and 6)  and snow/ice (13) are not present and need filled back in
# Change and Land_Use have all values, so just putting a very large number will cause it to skip it
missing_values = {'Change':[9999],
                  'Land_Cover':[2,6,13],
                  'Land_Use':[9999]
                 }
print(raw_lcms.aggregate_histogram('system:time_start').getInfo())
print(raw_lcms.aggregate_histogram('year').getInfo())
for product in products:
    product_title = product.replace('_',' ')
    raw_lcms_product = raw_lcms.filter(ee.Filter.eq('product',product))
    years = raw_lcms_product.aggregate_histogram('year').keys().getInfo()
    c = []
    for year in years:
        year = int(float(year))
        raw_lcms_product_yr = raw_lcms_product.filter(ee.Filter.eq('year',year)).mosaic()
        # Map.addLayer(raw_lcms_product_yr,{},f'Raw LCMS {product_title} {year}')
        
        max_prob_class = raw_lcms_product_yr.arrayArgmax().arrayGet(0).add(1).byte().rename([product])
        max_prob_class = ee.Image(max_prob_class.set(lcms_viz_dict))
        
        # Handle missing land cover types
        for missing_value in missing_values[product]:
            max_prob_class = max_prob_class.where(max_prob_class.gte(missing_value),max_prob_class.add(1))
        # print(max_prob_class.visualize().getInfo())
        # print(max_prob_class.getInfo())
        # Map.addLayer(max_prob_class,{'autoViz':True},f'Most Probable LCMS {product_title} {year}')
        c.append(max_prob_class.set('system:time_start',ee.Date.fromYMD(year,6,1).millis()))
        # print(product,year)
    c = ee.ImageCollection(c)
    Map.addTimeLapse(c,{'autoViz':True},f'Most Probable LCMS {product_title}')
Map.turnOnInspector()
Map.view()
print('Done')

{'1.0228896E12': 28, '1.0544256E12': 28, '1.086048E12': 28, '1.117584E12': 28, '1.14912E12': 28, '1.180656E12': 28, '1.2122784E12': 28, '1.2438144E12': 28, '1.2753504E12': 28, '1.3068864E12': 28, '1.3385088E12': 28, '1.3700448E12': 28, '1.4015808E12': 28, '1.4331168E12': 28, '1.4647392E12': 28, '1.4962752E12': 28, '1.5278112E12': 28, '1.5593472E12': 28, '1.5909696E12': 28, '1.6225056E12': 28, '1.6540416E12': 28, '4.86432E11': 42, '5.17968E11': 42, '5.49504E11': 42, '5.811264E11': 42, '6.126624E11': 42, '6.441984E11': 42, '6.757344E11': 42, '7.073568E11': 42, '7.388928E11': 42, '7.704288E11': 42, '8.019648E11': 30, '8.335872E11': 28, '8.651232E11': 28, '8.966592E11': 28, '9.281952E11': 28, '9.598176E11': 28, '9.913536E11': 28}
{'1985.0': 42, '1986.0': 42, '1987.0': 42, '1988.0': 42, '1989.0': 42, '1990.0': 42, '1991.0': 42, '1992.0': 42, '1993.0': 42, '1994.0': 42, '1995.0': 30, '1996.0': 28, '1997.0': 28, '1998.0': 28, '1999.0': 28, '2000.0': 28, '2001.0': 28, '2002.0': 28, '2003.0': 2

Done


In [31]:
Map.clearMap()
# Now lets export predicted assets
# First, we'll set up the study area and a tile to export across

studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
tileSize = 60000

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Set up years to apply models across
apply_years = list(range(1985,2022+1))

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))

Map.centerObject(studyArea)
# Map.view()

Adding layer: Tile Grid 60000m


In [ ]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_rawLCMSOutputs_collection, type = 'imageCollection')

print('done')